In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
data = pd.read_excel('../data/data.xlsx')
data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])
data = data.set_index('Unnamed: 0')
data.index.name = None
data.head()

,AAPL,MSFT,NVDA,AMZN,GOOG,META,BRK-B,TSLA,AVGO,WMT,...,SBER,ROSN,BTC-USD,GC%3DF,SI%3DF,PL%3DF,HG%3DF,PA%3DF,CL%3DF,NG%3DF
2016-01-01,23.834379,48.240696,0.789731,31.849501,36.917145,101.740501,130.75,14.894000,10.941828,17.191137,...,1.343893,3.441661,434.334015,1075.099976,13.817,883.200012,2.0705,542.950012,36.759998,2.334
2016-01-02,23.834379,48.240696,0.789731,31.849501,36.917145,101.740501,130.75,14.894000,10.941828,17.191137,...,1.343893,3.441661,433.437988,1075.099976,13.817,883.200012,2.0705,542.950012,36.759998,2.334
2016-01-03,23.834379,48.240696,0.789731,31.849501,36.917145,101.740501,130.75,14.894000,10.941828,17.191137,...,1.343893,3.441661,430.010986,1075.099976,13.817,883.200012,2.0705,542.950012,36.759998,2.334
2016-01-04,23.834379,48.240696,0.789731,31.849501,36.917145,101.740501,130.75,14.894000,10.941828,17.191137,...,1.343893,3.441661,433.091003,1075.099976,13.817,883.200012,2.0705,542.950012,36.759998,2.334
2016-01-05,23.237108,48.460758,0.802417,31.689501,36.953972,102.248108,131.25,14.895333,10.575768,17.599522,...,1.355960,3.471827,431.959991,1078.400024,13.949,888.599976,2.0885,535.099976,35.970001,2.325


In [3]:
pre = data.groupby([data.index.year, data.index.month]).agg(["first", "last"])

data_pct_m = pd.DataFrame(index = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.max()).values
                     , columns = data.columns)
for i in data.columns:
    data_pct_m[i] = ( (pre[i]['last'] - pre[i]['first']) / pre[i]['first'] ).values

print(data_pct_m.shape)
data_pct_m.head()

(72, 200)


,AAPL,MSFT,NVDA,AMZN,GOOG,META,BRK-B,TSLA,AVGO,WMT,...,SBER,ROSN,BTC-USD,GC%3DF,SI%3DF,PL%3DF,HG%3DF,PA%3DF,CL%3DF,NG%3DF
2016-01-31,-0.081791,0.000669,-0.094944,-0.091236,0.009629,0.116513,-0.013155,-0.127046,-0.041632,0.092092,...,-0.053811,0.036291,-0.150960,0.045546,0.034643,-0.014983,-0.007164,-0.078675,-0.121690,-0.057127
2016-02-29,0.008137,-0.063328,0.074207,-0.038778,-0.072114,-0.070988,0.042826,-0.025439,-0.026946,-0.017185,...,0.110019,0.067139,0.173274,0.093980,0.039570,0.074701,0.037533,-0.011962,0.067362,-0.204926
2016-03-31,0.084154,0.050399,0.087939,0.025214,0.036366,0.038973,0.036074,0.233002,0.119783,0.038174,...,0.139653,0.181896,-0.042273,0.003170,0.049345,0.043265,0.018678,0.087809,0.114535,0.124569
2016-04-30,-0.148044,-0.098135,-0.012817,0.115583,-0.073564,0.015940,0.015022,0.014801,-0.075665,-0.028236,...,0.184284,0.211440,0.072634,0.056319,0.179675,0.132795,0.051905,0.115937,0.237836,0.090320
2016-05-31,0.072536,0.059713,0.306836,0.069590,0.056345,0.004821,-0.040158,-0.075474,0.057478,0.058622,...,0.061050,-0.112104,0.175958,-0.060383,-0.097644,-0.095945,-0.074904,-0.124953,0.087245,0.096135


In [4]:
data_pct = data.pct_change().dropna()

pdata = data[:366]
pdata_pct = pdata.pct_change().dropna()
pdata_pct_m = data_pct_m[:12]

In [5]:
import random
from scipy import stats

In [6]:
def make_data_uniform(df):
    df = pd.DataFrame(df)
    us = []
    for col in df.columns:
        us.append((df[col] - min(df[col])) / (max(df[col] - min(df[col]))))
    us = pd.DataFrame(us).T
    us.columns = df.columns
    return us

In [7]:
#data slicer
indmin = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.min()).values
indmax = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.max()).values

#itg lists with frames
periods = []
cors = []

p_periods = []

for i in range(0, 61):
    #data slicer
    period = data.loc[indmin[i]:indmax[i+11]]
    period = make_data_uniform(period)
    period_corr = period.corr()
    
    cors.append(period_corr.values.astype(np.float32))
    periods.append(period.values.astype(np.float32))
    
    #choose 10 assets:max corr, 10 assets:min corr, 5 assets:randomly
    abs_corr = period_corr.abs().sum().sort_values()

    asset_labels = []
    asset_labels += abs_corr[:10].index.to_list()
    asset_labels += abs_corr.index[random.sample(range(11, len(data.columns)-10), 5)].to_list()
    asset_labels += abs_corr[-10:].index.to_list()

    p_periods.append(period[asset_labels].values.astype(np.float32))

In [8]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

from rpy2.robjects import pandas2ri

C:\Users\kirik\anaconda3\envs\rstudio\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [9]:
def get_copula_sims(df):
    pandas2ri.activate()

    # Загружаем необходимые R-пакеты
    copula = importr('copula')

    # Определяем R-функцию
    r_code = """
    fit_copulas <- function(data) {
      library(copula)
      
      nc = normalCopula(P2p(data), dim=ncol(data), dispstr="un")
      nc_sim = rCopula(n=1000, copula=nc)

      list(
        normal_cop_sims = nc_sim
      )
    }
    """
    ro.r(r_code)  # Загружаем функцию в R

    # Вызываем R-функцию из Python
    copula_fits = ro.r['fit_copulas'](df)

    # Извлекаем результаты
    norm_sims = copula_fits.rx2('normal_cop_sims')
    
    return norm_sims

In [10]:
from sklearn.covariance import empirical_covariance, shrunk_covariance, graphical_lasso
from sklearn.preprocessing import StandardScaler

In [11]:
from tqdm import tqdm
from IPython.display import clear_output

In [14]:
corrs_itg = []
corrs_gl = []

for period in tqdm(periods):
    myScaler = StandardScaler()
    p_scaled = myScaler.fit_transform(period)

    emp_cov = empirical_covariance(p_scaled, assume_centered=True)
    shrunk_cov = shrunk_covariance(emp_cov, shrinkage=0.8)
    gl_cov, _ = graphical_lasso(shrunk_cov, alpha=0.05)
    corrs_gl.append(gl_cov)
    
    sims = np.array(get_copula_sims(gl_cov))
    #corr_itg = stats.spearmanr(sims)
    corr_itg = pd.DataFrame(sims).corr(method='kendall').values
    corrs_itg.append(corr_itg)
    
    clear_output(wait=True)

100%|██████████| 61/61 [12:26<00:00, 12.24s/it]


In [ ]:
#corrs_itg = [i[0] for i in corrs_itg]

In [16]:
for i in range(len(corrs_gl)):
    np.savetxt('intermediate_data/glasso_cors/glasso_cors%s.gz' %i
               , corrs_gl[i]
               , delimiter=',')
    
for i in range(len(corrs_itg)):
    np.savetxt('intermediate_data/glasso_cors/glasso_kendall%s.gz' %i
               , corrs_itg[i]
               , delimiter=',')